In [111]:
import Pkg

Pkg.add("JLD2")

    Updating registry at `C:\Users\matsz\.julia\registries\General.toml`
   Resolving package versions...
   Installed FileIO ─ v1.16.3
   Installed JLD2 ─── v0.4.51
    Updating `C:\Users\matsz\.julia\environments\v1.10\Project.toml`
  [033835bb] + JLD2 v0.4.51
    Updating `C:\Users\matsz\.julia\environments\v1.10\Manifest.toml`
  [5789e2e9] + FileIO v1.16.3
  [033835bb] + JLD2 v0.4.51
Precompiling project...
  ✓ FileIO
  ✓ Plots → FileIOExt
  ✓ JLD2
  ✓ TaylorSeries → TaylorSeriesJLD2Ext
  4 dependencies successfully precompiled in 79 seconds. 538 already precompiled.


In [3]:
using Revise
using Turing, MCMCChains
using XLSX
using DataFrames, TimeSeries
using StatsPlots
using LinearAlgebra
using CSV
using StatsBase
include("utils.jl")
include("financial_bvar_temp.jl")
using .FinancialBVAR
using JLD2

In [45]:
data = TimeArray(CSV.File("usa_data_transformed.csv"),timestamp=:timestamp)
start = Date(1993,03,01)
end_d = Date(2023, 12, 31)
s_data = to(from(data, start),end_d)

124×6 TimeArray{Float64, 2, Date, Matrix{Float64}} 1993-03-01 to 2023-12-01
┌────────────┬─────────────────┬───────────────┬─────────────┬───────────────┬──
│            │ Total_Stock_CPI │ Price_10Y_CPI │ T_Bill_CPI  │ Dividend_Yeld │ ⋯
├────────────┼─────────────────┼───────────────┼─────────────┼───────────────┼──
│ 1993-03-01 │       0.0150658 │     0.0543577 │ -0.00250148 │     0.0281965 │ ⋯
│ 1993-06-01 │       0.0136547 │     0.0292271 │  0.00478562 │     0.0279908 │ ⋯
│ 1993-09-01 │       0.0342312 │       0.03506 │ -0.00169417 │     0.0270317 │ ⋯
│ 1993-12-01 │       0.0226641 │    -0.0219636 │  0.00410946 │     0.0266883 │ ⋯
│ 1994-03-01 │      -0.0571526 │    -0.0622961 │ 0.000425924 │     0.0285162 │ ⋯
│ 1994-06-01 │      0.00958316 │    -0.0293652 │  0.00338014 │     0.0285113 │ ⋯
│ 1994-09-01 │        0.026626 │   -0.00884948 │  0.00390608 │     0.0280574 │ ⋯
│ 1994-12-01 │      0.00488791 │  -0.000941161 │  0.00828514 │     0.0283289 │ ⋯
│     ⋮      │        ⋮        │ 

In [46]:
vmodel = VARModel(s_data)
scenarios = simulate(vmodel,60, 100_000)
returns_scenarios = scenarios[1:3,:,:]

3×100000×61 Array{Float64, 3}:
[:, :, 1] =
 0.121595   0.121595   0.121595   …  0.121595   0.121595   0.121595
 0.0651554  0.0651554  0.0651554     0.0651554  0.0651554  0.0651554
 0.0103466  0.0103466  0.0103466     0.0103466  0.0103466  0.0103466

[:, :, 2] =
  0.120726     0.0553505    0.0228392  …   0.0977781   -0.0382084
 -0.0153241   -0.0022677   -0.0803834     -0.00419044   0.028488
 -0.00366503   0.00501138  -0.0106339      0.00249935   0.00898509

[:, :, 3] =
  0.137978    0.0567325   0.0775392    0.0161274  …  0.0573017   0.00516247
 -0.0489355  -0.027689    0.0588387   -0.0186317     0.0352686   0.00466669
  0.0042137  -0.00479787  0.00615595   0.0106845     0.00645348  0.00615261

;;; … 

[:, :, 59] =
 -0.113561    -0.0644483   -0.0964931   …  -0.0472061    0.0432604
  0.0259695   -0.0373943   -0.00491977      0.0122742    0.0562046
 -0.00188306  -0.00639914  -0.00321069      0.000276954  0.00881693

[:, :, 60] =
 -0.0598416    -0.0480842  -0.00754302  …  -0.0095841   -0.12

In [47]:
includet("kmeans_moments_lattice.jl")
using .LatticeGeneration

In [8]:
s = sum(returns_scenarios[:,:,21:25]; dims=3)
s = dropdims(s; dims= 3)
mean(s[1,:,:])

5.077523642321002

In [49]:
returns_scenarios_ann = zeros(3,100_000,5)
for asset in 1:3
    returns_scenarios_ann[asset,:,:] = exp.(annualise(returns_scenarios[asset,:,2:end],12))
end
returns_scenarios_ann


3×100000×5 Array{Float64, 3}:
[:, :, 1] =
 1.55057  1.07996  1.65553  1.37401   …  1.87546   1.16741  1.03746
 1.07534  0.97992  1.02929  1.14199      0.999001  0.91502  0.961497
 1.01565  1.02956  1.01149  0.986451     1.08687   1.01029  1.06476

[:, :, 2] =
 1.1501    1.00336  1.46611   …  1.44351  1.32588  1.42396  0.958516
 1.03199   1.08714  1.03937      1.00556  1.08632  1.22259  1.14063
 0.965993  1.06949  0.993166     1.06287  1.0225   1.00911  1.02268

[:, :, 3] =
 1.1677   1.63675  0.726168  1.02879   …  0.704902  1.32088   1.6459
 1.22918  1.16335  0.974245  0.953223     0.890175  0.838488  1.04052
 0.94722  1.05511  0.883239  0.852686     0.952807  1.01876   1.00562

[:, :, 4] =
 1.23879   1.48356   1.04649   1.33438   …  0.994284  0.938806  1.2328
 0.758898  0.978688  1.05904   0.803775     1.0556    1.19577   1.12371
 0.890746  1.00392   0.943388  0.839643     0.994243  1.0119    1.04322

[:, :, 5] =
 1.04793   1.18585   1.39569   1.15692   …  1.59266  0.989368  0.784258


In [50]:
res = generate_lattice(returns_scenarios_ann, 9,5)


Lattice([1.0853933315390984 1.218288162475466 … 1.8355566846831484 1.4009515471553637; 0.94539648550184 0.9352273412884807 … 1.5265792998503565 1.2383234954409708; … ; 0.9191554431977321 1.154816879403226 … 1.3300414893432977 1.038763668706857; 0.9261635028461954 1.7917767202361508 … 1.1708527976718917 1.3322260375257169;;; 0.9765946310583116 1.1774415120338133 … 0.9291402188272971 0.8899189392863953; 1.2181816749731287 1.0059295736387661 … 1.0865583310648845 0.8873290097745962; … ; 1.1156717683855484 0.9243580513031538 … 1.100655126873907 1.0823987352357525; 1.109849303328651 0.9550545807263788 … 0.9243075236913665 1.0942922799016646;;; 1.0402671160866452 1.0463306482683974 … 1.0228155547306848 1.0562637488834923; 1.0101162372306656 0.9790255920467821 … 1.0822322232049224 0.9539788581889932; … ; 0.9764644843180704 1.0088132672878474 … 0.9857306066451239 1.0572844235140704; 0.9713402918750139 1.0025639047032668 … 1.0070728088493606 0.9795746837072669], [[1.0;;], [0.157 0.10318 … 0.0706

In [51]:
res.states

5×9×3 Array{Float64, 3}:
[:, :, 1] =
 1.08539   1.21829   1.11716  1.71048  1.48497  …  1.28611   1.83556  1.40095
 0.945396  0.935227  1.23291  1.02688  1.32241     1.13014   1.52658  1.23832
 1.72884   1.20305   1.13756  1.22819  1.04395     0.934721  1.31129  1.55612
 0.919155  1.15482   1.48862  1.0715   1.77658     0.998231  1.33004  1.03876
 0.926164  1.79178   1.03962  1.07935  1.41463     0.999444  1.17085  1.33223

[:, :, 2] =
 0.976595  1.17744   1.06451   1.03996   …  0.898513  0.92914   0.889919
 1.21818   1.00593   0.913769  1.05636      1.28548   1.08656   0.887329
 0.927796  0.875135  1.30499   0.911116     1.20834   1.09984   1.08863
 1.11567   0.924358  1.11646   1.31595      0.916218  1.10066   1.0824
 1.10985   0.955055  1.08491   1.318        0.911537  0.924308  1.09429

[:, :, 3] =
 1.04027   1.04633   0.998187  1.07006  …  1.00309   1.02282   1.05626
 1.01012   0.979026  1.02889   1.06323     1.08355   1.08223   0.953979
 0.991963  0.931532  1.08131   1.01325     

In [52]:
res.probs

6-element Vector{Matrix{Float64}}:
 [1.0;;]
 [0.157 0.10318 … 0.07065 0.12649]
 [0.09159235668789809 0.08452229299363058 … 0.15369426751592358 0.032292993630573245; 0.016379143244814886 0.1560379918588874 … 0.013956193060670672 0.3638301996510952; … ; 0.20438782731776362 0.21656050955414013 … 0.04203821656050955 0.03156404812455768; 0.1310775555379872 0.04316546762589928 … 0.16499328010119377 0.003952881650723377]
 [0.14910634936308878 0.2410388071492051 … 0.14357657746617952 0.03989335439913103; 0.10473428648502832 0.14836795252225518 … 0.17534394388993796 0.05887510115996763; … ; 0.04352767359927264 0.030117058756676893 … 0.0968291851346744 0.1301284236845096; 0.049712291293274216 0.14932553532685597 … 0.14706159796245638 0.03527969059522686]
 [0.20655737704918034 0.17154098360655737 … 0.10688524590163935 0.1318032786885246; 0.19810795770728992 0.12289000185494342 … 0.1390280096457058 0.05481357818586533; … ; 0.15993860322333078 0.17498081350729086 … 0.12087490406753645 0.04919416730

In [55]:
round.(res.states[1,:,:]; digits=4)

9×3 Matrix{Float64}:
 1.0854  0.9766  1.0403
 1.2183  1.1774  1.0463
 1.1172  1.0645  0.9982
 1.7105  1.04    1.0701
 1.485   1.053   1.0215
 1.2305  1.0272  1.0837
 1.2861  0.8985  1.0031
 1.8356  0.9291  1.0228
 1.401   0.8899  1.0563

In [57]:
round.(res.states[2,:,:]; digits=4)

9×3 Matrix{Float64}:
 0.9454  1.2182  1.0101
 0.9352  1.0059  0.979
 1.2329  0.9138  1.0289
 1.0269  1.0564  1.0632
 1.3224  1.1029  1.0045
 1.7456  0.9406  1.0122
 1.1301  1.2855  1.0835
 1.5266  1.0866  1.0822
 1.2383  0.8873  0.954

In [58]:
round.(res.probs[6]; digits=2)

9×9 Matrix{Float64}:
 0.11  0.11  0.04  0.03  0.2   0.04  0.18  0.14  0.17
 0.13  0.07  0.18  0.15  0.03  0.13  0.03  0.15  0.13
 0.1   0.06  0.23  0.14  0.03  0.15  0.04  0.17  0.08
 0.09  0.12  0.09  0.04  0.13  0.09  0.1   0.19  0.13
 0.19  0.03  0.18  0.1   0.05  0.07  0.12  0.17  0.08
 0.22  0.04  0.08  0.06  0.1   0.03  0.21  0.14  0.13
 0.16  0.07  0.04  0.04  0.17  0.03  0.21  0.11  0.17
 0.16  0.06  0.05  0.02  0.17  0.02  0.22  0.17  0.13
 0.08  0.09  0.18  0.14  0.03  0.2   0.02  0.15  0.11

In [59]:
save_object("scenario_grid.jld2", res)

In [60]:
grid = load_object("scenario_grid.jld2")

Lattice([1.0853933315390984 1.218288162475466 … 1.8355566846831484 1.4009515471553637; 0.94539648550184 0.9352273412884807 … 1.5265792998503565 1.2383234954409708; … ; 0.9191554431977321 1.154816879403226 … 1.3300414893432977 1.038763668706857; 0.9261635028461954 1.7917767202361508 … 1.1708527976718917 1.3322260375257169;;; 0.9765946310583116 1.1774415120338133 … 0.9291402188272971 0.8899189392863953; 1.2181816749731287 1.0059295736387661 … 1.0865583310648845 0.8873290097745962; … ; 1.1156717683855484 0.9243580513031538 … 1.100655126873907 1.0823987352357525; 1.109849303328651 0.9550545807263788 … 0.9243075236913665 1.0942922799016646;;; 1.0402671160866452 1.0463306482683974 … 1.0228155547306848 1.0562637488834923; 1.0101162372306656 0.9790255920467821 … 1.0822322232049224 0.9539788581889932; … ; 0.9764644843180704 1.0088132672878474 … 0.9857306066451239 1.0572844235140704; 0.9713402918750139 1.0025639047032668 … 1.0070728088493606 0.9795746837072669], [[1.0;;], [0.157 0.10318 … 0.0706

In [38]:
using  Test
@test grid.states[5,:,:] ≈ res.states[5,:,:]

Test Passed

In [41]:
res.states[3,:,3]

9-element Vector{Float64}:
 1.0143305473204354
 0.9854785572603496
 1.003604484079091
 1.0038924296581662
 1.0107697123646282
 0.9910509301591774
 1.0332650815623272
 1.0259226703370474
 1.034748568861909

#### TODO

- evaluate moments